In [ ]:
!pip install langchain langchain-community PyMuPDF faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Güncel kütüphanelerin kurulumu
!pip install -U langchain-ollama langchain-huggingface PyMuPDF faiss-cpu

# Colab içine Ollama kurulumu
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > ollama.log 2>&1 &
!ollama pull llama3


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [ ]:
import os
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

class AOFAsistan:
    def __init__(self, dosya_yolu):
        self.dosya_yolu = dosya_yolu
        self.loader = PyMuPDFLoader(dosya_yolu)
        # Use OllamaLLM from langchain_ollama as recommended
        self.model = OllamaLLM(model="llama3")
        # Use HuggingFaceEmbeddings from langchain_huggingface as recommended
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    def veri_isle(self):
        dokumanlar = self.loader.load()
        # Keep the corrected import for RecursiveCharacterTextSplitter
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
        self.parcalar = text_splitter.split_documents(dokumanlar)
        self.vector_db = FAISS.from_documents(self.parcalar, self.embeddings)
        print(f"Sistem hazır: {self.dosya_yolu} yüklendi.")

    def unite_analizi_yap(self):
        prompt = """
        Sen bir AÖF akademik asistanısın. Aşağıdaki ünite metnine dayanarak:
        1. Ünitedeki EN KRİTİK 5 kavramı açıkla.
        2. Sınavda çıkabilecek en önemli 3 noktayı belirt.
        3. Öğrenci için kısa bir çalışma özeti çıkar.

        Metin:
        """
        icerik = " ".join([d.page_content for d in self.parcalar[:5]])
        return self.model.invoke(prompt + icerik)

    def soru_sor(self, soru):
        ilgili_metinler = self.vector_db.similarity_search(soru, k=3)
        context = " ".join([m.page_content for m in ilgili_metinler])

        rag_prompt = f"""
        Bilgi: {context}
        Soru: {soru}
        Yukarıdaki bilgiye dayanarak soruyu bir öğretmen gibi açıkla.
        Eğer cevap metinde yoksa 'Bu bilgi ünite içinde yer almıyor' de.
        """
        return self.model.invoke(rag_prompt)

# --- KULLANIM ---
dosya = "/content/drive/MyDrive/sosyalpolitika/document (9).pdf"
asistan = AOFAsistan(dosya)
asistan.veri_isle()

print("\n--- ÜNİTE STRATEJİSİ ---")
print(asistan.unite_analizi_yap())

print("\n--- ÖĞRENCİ SORUSU TESTİ ---")
print(asistan.soru_sor("Küreselleşmenin sosyal politika üzerindeki olumsuz etkileri nelerdir?"))

Sistem hazır: /content/drive/MyDrive/sosyalpolitika/document (9).pdf yüklendi.

--- ÜNİTE STRATEJİSİ ---
As an academic assistant at AÖF, I'll be happy to help you with the following tasks:

**1. Identify the 5 most critical concepts in this unit:**

After analyzing the text, I've identified the following 5 most critical concepts:

1. **Sosyal politika**: The concept of social policy as a whole, which encompasses various government decisions and actions aimed at improving the well-being of citizens.
2. **Social justice**: The idea that social policy should aim to promote social fairness and equity, ensuring that everyone has access to basic needs such as education, healthcare, and employment.
3. **Class struggle**: The concept of class conflict, which is seen as a driving force behind social policy decisions and the need for social balance.
4. **Sanayi Devrimi (Industrial Revolution)**: The significant economic and technological changes that occurred in Europe during the 18th and 19th 